<a href="https://colab.research.google.com/github/pdkang/multi-agent/blob/main/01_Agentic_RAG_Notebook_pk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Dive Agentic Retrieval Augmented Generation

An Agentic RAG is required when we use reasoning to determine which action(s) to take and in which order to take them. Essentially we use agents instead of a LLM directly to accomplish a set of tasks which requires planning, multi step reasoning, tool use and/or learning over time. Agents give us agency!

Agency : The ability to take action or to choose what action to take

In the context of RAG, we can plug in agents to enhance the reasoning prior to selection of RAG pipelines, within a RAG pipeline for retrieval or reranking and finally for synthesising before we send out the response. This improves RAG to a large extent by automating complex workflows and decisions that are required for a non trivial RAG use case.

### Purpose of this Agentic RAG
This notebook presents a practical implementation of Agentic Retrieval-Augmented Generation (RAG)—a system where decision-making and tool selection are delegated to an intelligent agent before executing a response. Rather than passing every query through a static RAG pipeline, this system introduces agency—the ability to choose the best course of action depending on the nature of the query.

At the heart of this implementation is a router prompt, which classifies user queries into one of three categories:

- OpenAI documentation: Queries related to tools, APIs, or usage guidelines for OpenAI models
- 10-K financial reports: Questions requiring retrieval from company filings or financial datasets
- Live Internet search: Broader, current, or comparative queries that need web access

Once the query is classified, the system invokes a corresponding route handler:

- For OpenAI and 10-K queries, it retrieves relevant context from a vector database (Qdrant) using text embeddings, then applies a RAG-based response generator.
- For Internet queries, it fetches real-time information using a web-access API (ARES).

This approach is an example of Agentic RAG, where reasoning precedes retrieval and generation. By plugging in agents before and within the RAG pipeline, we make the system smarter and more adaptive. This allows us to:

- Automatically choose the right retrieval method based on context
- Combine structured knowledge with real-time search
- Scale RAG beyond trivial use cases by integrating multi-step decision logic

Importantly, no external agentic frameworks are used—this is a ground-up implementation that demonstrates how to build a lightweight but intelligent agentic system using only a language model, prompt engineering, and retrieval tools.

## Setup and Dependencies

In [2]:
# Install the necessary libraries
!pip install openai
!pip install qdrant_client
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 5.8 MB/s eta 0:00:00


In [3]:
# Import basic libraries
import requests             # Used for making HTTP requests (e.g., calling ARES API for live internet queries)
import json                 # For parsing and structuring JSON data (especially OpenAI and routing responses)

# Google Colab-specific (for securely handling API keys)
from google.colab import userdata  # To securely store and retrieve credentials in Colab

# OS operations
import os                   # Useful for accessing environment variables and managing paths

# OpenAI API client
from openai import OpenAI   # Official OpenAI client library to interface with GPT models for routing and generation

# Text processing
import re                   # Regular expressions for cleaning or preprocessing inputs (if needed)

# Optional visualization (for analysis/debugging purposes)
import matplotlib.pyplot as plt       # For displaying charts or visual debug outputs (e.g., embeddings visualizations)
import matplotlib.image as mpimg      # For loading/displaying images if needed (rare in RAG, but helpful in demos)

# Embedding models (used for text vectorization during retrieval)
from transformers import AutoTokenizer, AutoModel  # For loading custom transformer models if not using OpenAI embeddings

# Vector database client
from qdrant_client import QdrantClient   # Qdrant is used as the vector store to retrieve documents based on similarity


## 1. Defining the Internet Tool

First, we will define a tool function that enables our system to answer queries requiring real-time, internet-based information. Not all questions can be answered using static documents like OpenAI docs or financial filings—sometimes users ask about current trends, comparisons, or live updates.

To handle this, we introduce a live search capability using the **ARES API** by Traversaal.

### What is ARES API?  
ARES is a web-based tool that allows you to:

- Search the internet in real time.
- Get LLM-generated answers based on live search results.

This is particularly useful for questions about:

- Current events (e.g., *“Latest AI tools in 2025”*),
- Tech comparisons (e.g., *“Gemini vs GPT-4”*),
- General knowledge outside internal datasets.

Please generate the API key [here](https://api.traversaal.ai)


In [4]:
#loads ares api key from colab secrets
ares_api_key=userdata.get('ARES_API_KEY')

In [5]:
import requests  # For sending HTTP POST requests to the ARES API

def get_internet_content(user_query: str, action: str):
    """
    Fetches a response from the internet using ARES-API based on the user's query.

    This function serves as the tool invoked when the router classifies a query
    as requiring real-time information beyond internal datasets—i.e., "INTERNET_QUERY".
    It sends the query to a live search API (ARES) and returns the result.

    Args:
        user_query (str): The user's question that needs a live answer.
        action (str): Route type (always expected to be "INTERNET_QUERY").

    Returns:
        str: Response text generated using internet search or an error message.
    """
    print("Getting your response from the internet 🌐 ...")

    # API endpoint for the ARES live search tool
    url = "https://api-ares.traversaal.ai/live/predict"

    # Payload structure expected by the ARES API
    payload = {"query": [user_query]}

    # Authentication and content headers for API access
    headers = {
        "x-api-key": ares_api_key,  # Your secret API key (should be securely loaded from environment)
        "content-type": "application/json"
    }

    try:
        # Send the query to the ARES API and check for success
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()

        # Extract and return the main response text from the API's nested JSON
        return response.json().get('data', {}).get('response_text', "No response received.")

    # Handle HTTP-level errors (e.g., 400s or 500s)
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"

    # Handle general connection, timeout, or request formatting issues
    except requests.exceptions.RequestException as req_err:
        return f"Request error occurred: {req_err}"

    # Catch-all for any unexpected failure
    except Exception as err:
        return f"An unexpected error occurred: {err}"


In [14]:
print(get_internet_content("what are the best travel destinitions of 2025","INTERNET_QUERY")) #run internet function to test results

Getting your response from the internet 🌐 ...
Here are some of the best travel destinations for 2025:

1. **Budapest, Hungary** - Known for its stunning architecture and rich history, Budapest offers a vibrant cultural experience.
2. **Bukhara, Uzbekistan** - A city steeped in history, Bukhara is famous for its well-preserved medieval architecture.
3. **Charleston, South Carolina, USA** - Renowned for its historic charm, beautiful beaches, and Southern hospitality.
4. **Inverness and the Flow Country, Scotland** - A picturesque area known for its natural beauty and proximity to Loch Ness.
5. **Seoul, South Korea** - A bustling metropolis that blends modernity with tradition, offering a unique cultural experience.
6. **Kathmandu, Nepal** - The gateway to the Himalayas, known for its rich culture and history.
7. **Cusco, Peru** - The historic capital of the Inca Empire, offering access to Machu Picchu and rich cultural experiences.
8. **Bangkok, Thailand** - A vibrant city known for its 

## 2. Router Query Function — Giving the Agent Its Brain

In this step, we will define the router function, which plays a critical role in our Agentic RAG system.

### What is a Router?

A router is like the decision-making brain of our assistant.

Before trying to answer a user's question, the system first needs to figure out:

> “Where should I go to find the right answer?”

To make this decision, we use the OpenAI GPT model. We provide it with a detailed system prompt that explains how to classify the user's question into one of these categories:

- **OPENAI_QUERY** → Questions about OpenAI tools, APIs, models, or documentation.
- **10K_DOCUMENT_QUERY** → Questions about companies, financial filings, or analysis based on 10-K reports.
- **INTERNET_QUERY** → Anything else that likely requires real-time or general web information.

### What does the function do?

- Sends the user's question to the OpenAI API.
- Receives a JSON response containing:
  - `action`: The category the query belongs to.
  - `reason`: A short explanation for the decision.
  - `answer`: (Optional) A quick response if it’s simple enough (left blank for internet queries).
- Parses the response and returns it as a Python dictionary.

### Why is this important?

This router gives the system agency—the ability to decide which knowledge source to use. It’s what makes this pipeline agentic, not just static.

Without the router, every query would follow the same path. With it, we can:

- Dynamically switch between tools and data sources.
- Handle different types of user questions intelligently.
- Avoid wasting resources on unnecessary steps.


In [6]:
# Securely retrieve the OpenAI API key from Colab's user data store
# This avoids hardcoding sensitive credentials directly in the notebook
openai_api_key = userdata.get('OPENAI_API_KEY')

# Initialize the OpenAI client with the retrieved API key
# This client will be used for:
# - Query classification via the router prompt
# - Potentially generating responses from retrieved context
openaiclient = OpenAI(api_key=openai_api_key)


In [7]:
from openai import OpenAIError

def route_query(user_query: str):
    """
    Routes a user query to the appropriate source (OpenAI, 10k, or Internet).

    This function acts as the 'brain' of the agentic RAG system. It uses an LLM (GPT-4)
    to reason about the nature of the user's query and classify it into one of three routes:

    - "OPENAI_QUERY": For queries about OpenAI tools, models, or documentation
    - "10K_DOCUMENT_QUERY": For finance-related questions requiring 10-K filings
    - "INTERNET_QUERY": For open-ended, trending, or external-topic queries

    The LLM returns a structured JSON with:
    - action: selected route
    - reason: rationale behind routing decision
    - answer: a short answer (if applicable)

    Returns:
        dict: Classification output containing route, reasoning, and (optional) short answer.
    """

    # Construct a system prompt that instructs the model to act as a strict query router.
    # It explains the criteria for each classification and enforces a fixed response format.
    router_system_prompt =f"""
    As a professional query router, your objective is to correctly classify user input into one of three categories based on the source most relevant for answering the query:

    1. "OPENAI_QUERY": If the user's query appears to be answerable using information from OpenAI's official documentation, tools, models, APIs, or services (e.g., GPT, ChatGPT, embeddings, moderation API, usage guidelines).

    2. "10K_DOCUMENT_QUERY": If the user's query pertains to a collection of documents from the 10k annual reports, datasets, or other structured documents, typically for research, analysis, or financial content.

    3. "INTERNET_QUERY": If the query is neither related to OpenAI nor the 10k documents specifically, or if the information might require a broader search (e.g., news, trends, tools outside these platforms), route it here.

    Your decision should be made by assessing the domain of the query.

    Always respond in this valid JSON format:
    {{
        "action": "OPENAI_QUERY" or "10K_DOCUMENT_QUERY" or "INTERNET_QUERY",
        "reason": "brief justification",
        "answer": "AT MAX 5 words answer. Leave empty if INTERNET_QUERY"
    }}

    EXAMPLES:

    - User: "How to fine-tune GPT-3?"
    Response:
    {{
        "action": "OPENAI_QUERY",
        "reason": "Fine-tuning is OpenAI-specific",
        "answer": "Use fine-tuning API"
    }}

    - User: "Where can I find the latest financial reports for the last 10 years?"
    Response:
    {{
        "action": "10K_DOCUMENT_QUERY",
        "reason": "Query related to annual reports",
        "answer": "Access through document database"
    }}

    - User: "Top leadership styles in 2024"
    Response:
    {{
        "action": "INTERNET_QUERY",
        "reason": "Needs current leadership trends",
        "answer": ""
    }}

    - User: "What's the difference between ChatGPT and Claude?"
    Response:
    {{
        "action": "INTERNET_QUERY",
        "reason": "Cross-comparison of different providers",
        "answer": ""
    }}

    Strictly follow this format for every query, and never deviate.
    User: {user_query}
    """


    try:
        # Query the GPT-4 model with the router prompt and user input
        response = openaiclient.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "system", "content": router_system_prompt}]
        )

        # Extract and parse the model's JSON response
        task_response = response.choices[0].message.content
        parsed_response = json.loads(task_response)
        return parsed_response

    # Handle OpenAI API errors (e.g., rate limits, authentication)
    except OpenAIError as api_err:
        return {
            "action": "INTERNET_QUERY",
            "reason": f"OpenAI API error: {api_err}",
            "answer": ""
        }

    # Handle case where model response isn't valid JSON
    except json.JSONDecodeError as json_err:
        return {
            "action": "INTERNET_QUERY",
            "reason": f"JSON parsing error: {json_err}",
            "answer": ""
        }

    # Catch-all for any other unforeseen issues
    except Exception as err:
        return {
            "action": "INTERNET_QUERY",
            "reason": f"Unexpected error: {err}",
            "answer": ""
        }


In [8]:
route_query("what is the revenue of uber in 2021?")


{'action': '10K_DOCUMENT_QUERY',
 'reason': 'Revenues can be found in annual reports',
 'answer': "Refers to Uber's annual report"}

In [ ]:
route_query("What was Lyft's revenue growth rate in 2024 compared to 2023?")

{'action': '10K_DOCUMENT_QUERY',
 'reason': "Need to study Lyft's annual reports",
 'answer': "Check Lyft's 10K reports"}

In [ ]:
route_query("what was uber revenue in 2024?")

{'action': '10K_DOCUMENT_QUERY',
 'reason': "Information likely available in Uber's 10K report",
 'answer': ''}

In [9]:
route_query("What is the purpose of the OpenAI moderation API?")

{'action': 'OPENAI_QUERY',
 'reason': "The query is about OpenAI's product",
 'answer': 'Filters inappropriate content'}

## 3. Setting Up Qdrant Vector Database for Agentic RAG
In this step, we are connecting our agent to a pre-built vector database using Qdrant—a tool used to store and search document embeddings (numerical representations of text).

What Are We Doing?
We are loading an existing Qdrant database that was downloaded from a GitHub repository. This database already contains:

- Vectorized OpenAI documentation
- Vectorized 10-K financial filings

By loading this saved data:

- We save time (no need to re-embed the documents)
- We enable fast similarity search to retrieve relevant text chunks

This setup allows our system to perform semantic search, meaning it can understand the meaning of the user query and match it with the most relevant pieces of information stored in the database.


### Why This Matters in Agentic RAG
Once the router decides that the query should go to the OpenAI docs or the 10-K reports, our system uses Qdrant to:

- Search for the most relevant pieces of text
- Pass those to the model to generate a grounded answer

So, this step is essential to support retrieval-augmented generation (RAG) within our agentic flow.

#Data Sources:

**10K Database: Lyft 2024 & Uber 2021 SEC filings**

**OpenAI Docs: Official OpenAI documentation**

For lecture demo purposes, the vecitr database has already been created and hosted on Github which we will clone here. In order to create your own embeddings, the notebook and data will be hosted and shared on github

In [10]:
# Clone the project repository that contains prebuilt vector data (e.g., Qdrant collections)
# This includes document embeddings and configurations needed for retrieval (10-K, OpenAI docs)

!git clone https://github.com/hamzafarooq/multi-agent-course.git


Cloning into 'multi-agent-course'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 120 (delta 28), reused 35 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (120/120), 4.44 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [11]:
# 🗄️ Initializing Qdrant client with local path to vector database
# The path points to prebuilt Qdrant collections (10-K and OpenAI docs) cloned from the repository
# This enables fast, local retrieval of relevant document chunks based on semantic similarity
client = QdrantClient(path="/content/multi-agent-course/Module_1/Agentic_RAG/qdrant_data")


## 4. Building the Retriever and RAG for Vector Databases
In this section, we build the core logic that allows our agent to find relevant documents and generate grounded answers using them.

###Step 1: Import the Embedding Model
We start by importing the nomic-ai/nomic-embed-text-v1.5 model from Hugging Face. This model is used to convert any text (such as a user query) into a dense vector, known as an embedding. These embeddings capture the semantic meaning of text, allowing us to later compare and retrieve similar documents.


In [12]:
# Load the tokenizer and embedding model from Hugging Face
# This model converts raw text into dense vector representations (embeddings)
# Used for similarity search in Qdrant during document retrieval
text_tokenizer = AutoTokenizer.from_pretrained("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
text_model = AutoModel.from_pretrained("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

def get_text_embeddings(text):
    """
    Converts input text into a dense embedding using the Nomic embedding model.
    These embeddings are used to query Qdrant for semantically relevant document chunks.

    Args:
        text (str): The input text or query from the user.

    Returns:
        np.ndarray: A fixed-size vector representing the semantic meaning of the input.
    """
    # Tokenize and prepare input for the model
    inputs = text_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Forward pass to get model outputs
    outputs = text_model(**inputs)

    # Take the mean across all token embeddings to get a single vector (pooled representation)
    embeddings = outputs.last_hidden_state.mean(dim=1)

    # Convert to NumPy array and detach from computation graph
    return embeddings[0].detach().numpy()

# Example usage: Generate and preview the embedding of a test sentence
text = "This is a test sentence."
embeddings = get_text_embeddings(text)
print(embeddings[:5])  # Print first 5 dimensions for inspection


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hf_nomic_bert.py:   0%|          | 0.00/104k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

[ 1.2799692   0.40158355 -3.5162656  -0.3981321   1.5919138 ]


### Step 2: Define the Embedding Function
We then define a function get_text_embeddings() which:

- Tokenizes the input text
- Runs it through the model
- Computes the average of all token embeddings
- Returns a single vector that represents the full sentence

This vector will be used to query Qdrant to find the most relevant document chunks based on similarity.

In [13]:
def rag_formatted_response(user_query: str, context: list):
    """
    Generate a response to the user query using the provided context,
    with article references formatted as [1][2], etc.

    This function performs the final step in the RAG pipeline—synthesizing an answer
    from retrieved document chunks (context). It prompts the model to generate a
    grounded response, explicitly citing sources using a reference format.

    Args:
        user_query (str): The user's original question.
        context (list): List of text chunks retrieved from Qdrant (10-K or OpenAI docs).

    Returns:
        str: A generated response grounded in the retrieved context, with numbered citations.
    """

    # Construct a RAG prompt that includes both:
    # 1. The user's query
    # 2. The supporting context documents
    # The prompt instructs the model to answer using only the provided context,
    # and to include citations like [1], [2], etc. based on chunk IDs or order.
    rag_prompt = f"""
       Based on the given context, answer the user query: {user_query}\nContext:\n{context}
       and employ references to the ID of articles provided [ID], ensuring their relevance to the query.
       The referencing should always be in the format of [1][2]... etc. </instructions>
    """

    #  Call GPT-4o to generate the response using the RAG-style prompt
    response = openaiclient.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": rag_prompt},
        ]
    )

    # Return the model's generated answer
    return response.choices[0].message.content


### Step 3: Define the RAG Response Generator
After retrieving relevant text chunks from Qdrant, we use the rag_formatted_response() function to generate a final answer. This function:

- Takes the user query and the retrieved document chunks
- Builds a prompt that asks the language model (GPT-4o) to answer the question using only the provided context
- Instructs the model to include references like [1], [2] for traceability

This ensures the output is not only informative but also grounded in actual retrieved data.

Together, these two functions lay the foundation for combining retrieval (from vector DB) and generation (from LLM) — the two pillars of a RAG system.



In [26]:
def retrieve_and_response(user_query: str, action: str):
    """
    Retrieves relevant text chunks from the appropriate Qdrant collection
    based on the query type, then generates a response using RAG.

    This function powers the retrieval and response generation pipeline
    for queries that are classified as either OPENAI-related or 10-K related.
    It uses semantic search to fetch relevant context from a Qdrant vector store
    and then generates a response using that context via a RAG prompt.

    Args:
        user_query (str): The user's input question.
        action (str): The classification label from the router (e.g., "OPENAI_QUERY", "10K_DOCUMENT_QUERY").

    Returns:
        str: A model-generated response grounded in retrieved documents, or an error message.
    """

    # Define mapping of routing labels to their respective Qdrant collections
    collections = {
        "OPENAI_QUERY": "opnai_data",           # Collection of OpenAI documentation embeddings
        "10K_DOCUMENT_QUERY": "10k_data"        # Collection of 10-K financial document embeddings
    }

    try:
        # Ensure that the provided action is valid
        if action not in collections:
            return "Invalid action type for retrieval."

        # Step 1: Convert the user query into a dense vector (embedding)
        try:
            query = get_text_embeddings(user_query)
        except Exception as embed_err:
            return f"Embedding error: {embed_err}"  # Fail early if embedding fails

        # Step 2: Retrieve top-matching chunks from the relevant Qdrant collection
        try:
            text_hits = client.query_points(
                collection_name=collections[action],  # Choose the right collection based on routing
                query=query,                          # The embedding of the user's query
                limit=3                               # Fetch top 3 relevant chunks
            ).points
        except Exception as qdrant_err:
            return f"Vector DB query error: {qdrant_err}"  # Handle Qdrant access issues

        # Extract the raw content from the retrieved vector hits
        contents = [point.payload['content'] for point in text_hits]

        # If no relevant content is found, return early
        if not contents:
            return "No relevant content found in the database."

        # Step 3: Pass the retrieved context to the RAG model to generate a response
        try:
            response = rag_formatted_response(user_query, contents)
            return response
        except Exception as rag_err:
            return f"RAG response error: {rag_err}"  # Handle generation failures

    # Catch any unforeseen errors in the overall process
    except Exception as err:
        return f"Unexpected error: {err}"


# 4. Analyze, route, process compund query

In [33]:
def route_query(user_query: str):
    """
    Routes a user query to the appropriate source (OpenAI, 10k, or Internet).

    This function acts as the 'brain' of the agentic RAG system. It uses GPT-4
    to reason about the nature of the user's query and classify it into one of three routes.

    Args:
        user_query (str): The user's input question.

    Returns:
        dict: Classification output containing route, reasoning, and (optional) short answer.
    """
    # Construct router system prompt
    router_system_prompt = f"""
    As a professional query router, your objective is to correctly classify user input into one of three categories based on the source most relevant for answering the query:

    1. "OPENAI_QUERY": If the user's query appears to be answerable using information from OpenAI's official documentation, tools, models, APIs, or services.

    2. "10K_DOCUMENT_QUERY": If the user's query pertains to a collection of documents from the 10k annual reports, datasets, or other structured documents.

    3. "INTERNET_QUERY": If the query is neither related to OpenAI nor the 10k documents specifically, or if the information might require a broader search.

    Always respond in this valid JSON format:
    {{
        "action": "OPENAI_QUERY" or "10K_DOCUMENT_QUERY" or "INTERNET_QUERY",
        "reason": "brief justification",
        "answer": "AT MAX 5 words answer. Leave empty if INTERNET_QUERY"
    }}

    User: {user_query}
    """

    try:
        # Query the GPT-4 model with the router prompt
        response = openaiclient.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "system", "content": router_system_prompt}]
        )

        # Extract and parse the model's JSON response
        task_response = response.choices[0].message.content
        parsed_response = json.loads(task_response)
        return parsed_response

    except Exception as err:
        # Default fallback if routing fails
        return {
            "action": "INTERNET_QUERY",
            "reason": f"Routing error: {err}",
            "answer": ""
        }



def analyze_compound_query(user_query):
    """
    Enhanced compound query analyzer that better handles financial queries with multiple companies.
    """
    # First check for OpenAI API + financial query
    if "openai" in user_query.lower() and " and " in user_query.lower():
        financial_terms = ["revenue", "profit", "earnings", "financial", "income", "sales"]
        has_financial = any(term in user_query.lower() for term in financial_terms)

        if has_financial:
            # Split by " and "
            parts = user_query.split(" and ", 1)
            if len(parts) == 2:
                return parts

    # Check for financial terms
    financial_terms = ["revenue", "profit", "earnings", "financial", "income", "sales"]
    is_financial = any(term in user_query.lower() for term in financial_terms)

    # For financial queries, check for multiple companies
    if is_financial:
        companies = ["uber", "lyft", "amazon", "google", "microsoft", "apple", "tesla", "facebook", "meta"]
        found_companies = []

        for company in companies:
            if company in user_query.lower():
                found_companies.append(company)

        # If multiple companies found in a financial query, it's definitely compound
        if len(found_companies) >= 2:
            print(f"Financial query with companies: {found_companies}")

            # Extract year from the query
            year_match = re.search(r'\b(20\d\d)\b', user_query)
            year = year_match.group(1) if year_match else "2024"  # Default to 2024 if no year found

            # Create separate queries for each company
            sub_queries = []
            for company in found_companies:
                sub_query = f"What was {company.capitalize()} revenue in {year}?"
                sub_queries.append(sub_query)

            print(f"Split into company-specific queries: {sub_queries}")
            return sub_queries

    # If not a multi-company financial query, try the LLM-based approach
    compound_check_prompt = f"""
    You are a query analyzer that must determine if a query contains multiple distinct questions.

    STRICT RULES:
    1. If a query asks about MULTIPLE COMPANIES' financial data, it is ALWAYS compound.
    2. If a query asks about OpenAI APIs AND company financial data, it is ALWAYS compound.
    3. A query comparing different topics is NOT compound.

    The query: "{user_query}"

    Return ONLY a JSON object with this exact structure:
    {{"is_compound": true/false, "questions": ["question1", "question2", ...]}}

    If is_compound is true, the questions array MUST contain at least 2 elements.
    """

    try:
        compound_check_response = openaiclient.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": compound_check_prompt}],
            response_format={"type": "json_object"}  # Force JSON response
        )

        # response_text = compound_check_response.choices[0].message.content
        # print(f"Compound analysis response: {response_text}")

        # Parse the result
        result = json.loads(response_text)

        if result.get("is_compound", False) and len(result.get("questions", [])) >= 2:
            questions = result.get("questions")
            # print(f"LLM-based compound query detection: {questions}")
            return questions

    except Exception as e:
        # print(f"Error in LLM compound analysis: {e}")
        pass

    # If all detection methods fail, return original query
    return [user_query]


def route_and_verify_query(query):
    """
    Routes a single query and verifies document availability.
    """
    # First check if this is a financial query about a specific company
    try:
        financial_keywords = ["revenue", "earnings", "financial", "profit", "income", "sales"]
        is_financial_query = any(keyword in query.lower() for keyword in financial_keywords)

        if is_financial_query:
            # Extract company and year
            entity_prompt = f"""
            Extract ONLY the company name and year from this financial query.
            Return ONLY a JSON object: {{"company": "name", "year": "YYYY"}}

            Query: {query}
            """

            entity_response = openaiclient.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "system", "content": entity_prompt}],
                response_format={"type": "json_object"}
            )

            response_text = entity_response.choices[0].message.content
            # print(f"Entity extraction response: {response_text}")

            entities = json.loads(response_text)
            company = entities.get("company", "").lower().strip()
            year = entities.get("year", "").strip()

            # print(f"Extracted company: '{company}', year: '{year}'")

            # Define available company-year combinations
            available_data = [
                {"company": "uber", "year": "2021"},
                {"company": "lyft", "year": "2024"}
            ]

            # Check for exact matches in our database
            for entry in available_data:
                entry_company = entry["company"].lower()
                entry_year = entry["year"]

                # Normalize company names
                company_normalized = company.replace("'s", "").strip()
                entry_company_normalized = entry_company.replace("'s", "").strip()

                company_match = (
                    company_normalized == entry_company_normalized or
                    entry_company_normalized in company_normalized
                )
                year_match = entry_year == year

                # print(f"Comparing: '{company_normalized}' with '{entry_company_normalized}', match: {company_match}")
                # print(f"Comparing year: '{year}' with '{entry_year}', match: {year_match}")

                if company_match and year_match:
                    # print(f"MATCH FOUND: Financial data for {entry_company} {entry_year} exists")

                    # Route to 10K_DOCUMENT_QUERY
                    return {
                        "query": query,
                        "action": "10K_DOCUMENT_QUERY",
                        "reason": f"Financial data for {entry_company} {entry_year} found in database",
                        "verified": True
                    }

            # If no match, route to internet
            if company and year:
                # print(f"NO MATCH: Financial data for {company} {year} not found in database")
                return {
                    "query": query,
                    "action": "INTERNET_QUERY",
                    "reason": f"No financial data for {company} {year} in database",
                    "verified": True
                }
    except Exception as e:
        # print(f"Error in financial query check: {e}")
        pass

    # Continue with general routing if it's not a financial query or we couldn't determine
    try:
        route_result = route_query(query)
        action = route_result.get("action")
        reason = route_result.get("reason")

        # For OpenAI queries, we can trust the router
        if action == "OPENAI_QUERY":
            return {
                "query": query,
                "action": action,
                "reason": reason,
                "verified": True
            }

        # For all other cases, use the original route
        return {
            "query": query,
            "action": action,
            "reason": reason,
            "verified": True
        }
    except Exception as e:
        # Default to internet query for any errors
        return {
            "query": query,
            "action": "INTERNET_QUERY",
            "reason": f"Routing error: {e}",
            "verified": True
        }



def process_multi_route_query(routes_info):
    """
    Processes multiple routes for compound queries.

    Args:
        routes_info (list): List of route information dictionaries

    Returns:
        list: List of results from processing each route
    """
    results = []

    for route_info in routes_info:
        query = route_info["query"]
        action = route_info["action"]

        try:
            route_function = routes.get(action)
            if route_function:
                result = route_function(query, action)
                results.append({
                    "query": query,
                    "action": action,
                    "result": result
                })
            else:
                results.append({
                    "query": query,
                    "action": action,
                    "result": f"Unsupported action: {action}"
                })
        except Exception as e:
            results.append({
                "query": query,
                "action": action,
                "result": f"Error processing: {str(e)}"
            })

    return results

def format_multi_results(results):
    """
    Formats multiple results into a cohesive response.

    Args:
        results (list): List of query results

    Returns:
        str: Formatted response combining all results
    """
    if len(results) == 1:
        # Single result, just return it
        return results[0]["result"]

    # Multiple results, format them together
    response = "Here are the answers to your questions:\n\n"

    for i, result in enumerate(results):
        response += f"Question {i+1}: {result['query']}\n"
        response += f"Answer: {result['result']}\n\n"

    return response

# 5. Putting It All Together: Running the Agentic RAG
In this final step, we combine everything into a single function that controls the entire Agentic RAG workflow. The agentic_rag() function acts as the main orchestrator of the system.

Here’s what it does:

- Prints the user's query for reference.
- Uses the router function (powered by GPT) to decide which type of data source to use:
  - OpenAI documentation
  - 10-K financial reports
- Internet search
- Calls the correct function based on the route:
- If it’s an OpenAI or 10-K query, it retrieves data from Qdrant and generates a RAG response.
- If it’s an Internet query, it uses the ARES API to fetch live information.
- Displays the final response, neatly formatted in the console.

This step brings the agentic loop full circle—from understanding the question, reasoning about where to search, to finally responding with the best possible answer.

In [16]:
# Dictionary that maps the route labels (decided by the router) to their respective functions
# Each type of query is handled differently:
# - OPENAI_QUERY and 10K_DOCUMENT_QUERY use document retrieval + RAG
# - INTERNET_QUERY uses a web search API

routes = {
    "OPENAI_QUERY": retrieve_and_response,
    "10K_DOCUMENT_QUERY": retrieve_and_response,
    "INTERNET_QUERY": get_internet_content,
}

def agentic_rag(user_query: str):
    """
    Enhanced main function that handles compound queries and document availability.
    """
    # Terminal color codes
    CYAN = "\033[96m"
    GREY = "\033[90m"
    BOLD = "\033[1m"
    RESET = "\033[0m"

    try:
        # Print the user's original question
        print(f"{BOLD}{CYAN}👤 User Query:{RESET} {user_query}\n")

        # Step 1: Check if this is a compound query with our enhanced detector
        sub_queries = analyze_compound_query(user_query)
        is_compound = len(sub_queries) > 1

        if is_compound:
            print(f"{GREY}📍 Compound Query Detected: {len(sub_queries)} questions{RESET}\n")

        # Step 2: Process each sub-query independently
        all_results = []

        for i, query in enumerate(sub_queries):
            # Get routing for this specific sub-query
            route_info = route_and_verify_query(query)

            # Display routing information
            if is_compound:
                print(f"{GREY}Query {i+1}: {query}")
                print(f"Selected Route: {route_info['action']}")
                print(f"Reason: {route_info['reason']}")
                print(f"⚙️ Processing query {i+1}...{RESET}\n")
            else:
                print(f"{GREY}📍 Selected Route: {route_info['action']}")
                print(f"📝 Reason: {route_info['reason']}")
                print(f"⚙️ Processing query...{RESET}\n")

            # Process this sub-query
            try:
                action = route_info['action']
                route_function = routes.get(action)

                if route_function:
                    result = route_function(query, action)
                    all_results.append({
                        "query": query,
                        "action": action,
                        "result": result
                    })
                else:
                    all_results.append({
                        "query": query,
                        "action": action,
                        "result": f"Unsupported action: {action}"
                    })
            except Exception as e:
                all_results.append({
                    "query": query,
                    "action": action,
                    "result": f"Error processing: {str(e)}"
                })

        # Step 3: Format and display results
        print(f"{BOLD}{CYAN}🤖 BOT RESPONSE:{RESET}\n")

        if is_compound:
            # Format results for compound queries
            combined_response = ""

            for i, result_info in enumerate(all_results):
                query_part = result_info['query']
                result_part = result_info['result']

                combined_response += f"Question {i+1}: {query_part}\n"
                combined_response += f"Answer: {result_part}\n\n"

            print(combined_response)
        else:
            # Single result for non-compound queries
            print(f"{all_results[0]['result']}\n")

    except Exception as err:
        print(f"{BOLD}{CYAN}🤖 BOT RESPONSE:{RESET}\n")
        print(f"Unexpected error occurred: {err}\n")


In [31]:

agentic_rag("what was uber revenue in 2021?")

👤 User Query: what was uber revenue in 2021?

📍 Selected Route: 10K_DOCUMENT_QUERY
📝 Reason: Financial data for uber 2021 found in database
⚙️ Processing query...

🤖 BOT RESPONSE:

Uber's revenue in 2021 was $17,455 million [1].



In [18]:
agentic_rag("what was lyft revenue in 2024?")

👤 User Query: what was lyft revenue in 2024?

📍 Selected Route: 10K_DOCUMENT_QUERY
📝 Reason: The revenue of Lyft in 2024 can be found in their annual 10k report for that year.
⚙️ Processing query...

🤖 BOT RESPONSE:

Lyft's revenue for the year ended December 31, 2024, was $5,786,016,000, which represents a 31% increase compared to the previous year [1].



In [19]:
agentic_rag("what was uber revenue in 2024?")

👤 User Query: what was uber revenue in 2024?

📍 Selected Route: INTERNET_QUERY
📝 Reason: The data about Uber's revenue in 2024 is not available in OpenAI's documentation or in a 10k documents collection, and would require an internet search.
⚙️ Processing query...

Getting your response from the internet 🌐 ...
🤖 BOT RESPONSE:

Uber Technologies generated just under 44 billion U.S. dollars in net revenue in 2024. 

Here are some additional details regarding Uber's financial performance in 2024:

1. **Total Revenue**: 
   - The reported revenue for Uber in 2024 was approximately $44.0 billion, reflecting an 18% increase from the previous year (2023).
   - Other sources also confirm similar figures, with reports indicating revenues around $43.97 billion.

2. **Quarterly Breakdown**:
   - **Q1 2024**: Revenue grew 15% year-over-year to $10.1 billion.
   - **Q2 2024**: Revenue reached $3.3 billion, with delivery revenue growing 8% year-over-year.
   - **Q3 2024**: Revenue was reported at $1

In [20]:
agentic_rag("List me down new LLMs")

👤 User Query: List me down new LLMs

📍 Selected Route: INTERNET_QUERY
📝 Reason: The query is contextual and not specific to OpenAI or 10K documents. It could refer to programs, regulation, or something else related to LLMs (possibly referring to Master of Laws). An internet search would give the most comprehensive and updated results.
⚙️ Processing query...

Getting your response from the internet 🌐 ...
🤖 BOT RESPONSE:

1. **New Large Language Models in 2023:**
   - 2023 has been a significant year for the development of large language models (LLMs), with numerous new models emerging at a rapid pace.
   - The year saw a rise in small LLMs, particularly decoder-style transformers, with new pretrained models being released monthly.
   - Notable models include:
     - **GPT-4**: A multimodal model that has gained attention for its capabilities.
     - **BLOOM**: An open-access model that has been highlighted for its accessibility.
   - The landscape of LLMs in 2023 reflects a diverse rang

In [21]:
agentic_rag("how to work with chat completions?")

👤 User Query: how to work with chat completions?

📍 Selected Route: OPENAI_QUERY
📝 Reason: The question queries the operation of chat completions, which is covered in OpenAI's API and documentation.
⚙️ Processing query...

🤖 BOT RESPONSE:

To work with chat completions using the OpenAI API, you can follow these steps:

1. **Retrieve Chat Messages**: Use the endpoint to get the list of chat messages from a specific chat completion. You can specify parameters such as `completion_id`, the number of messages to retrieve (`limit`), and the sort order (`order`). For example, to send a request, you use:
   ```bash
   curl https://api.openai.com/v1/chat/completions/chat_abc123/messages \
     -H "Authorization: Bearer $OPENAI_API_KEY" \
     -H "Content-Type: application/json"
   ```

2. **Get Chat Completion**: You can fetch a specific chat completion object based on its ID (`completion_id`). This will give you details about the completion such as the model used, tokens utilized, and the full

#Assignment: Implement sub-query division

In our current agentic retrieval-augmented generation (RAG) setup, there's a key limitation: when a user submits a query that contains multiple distinct questions phrased as a single input, the system treats it as a single unified search. As a result, the retrieval engine performs only one operation on the vector databases or external tools, which often leads to incomplete or less relevant results.


To address this, the assignment introduces a new functionality called subquery division. This involves breaking down complex, compound queries into multiple, focused subqueries. Each subquery is processed independently, allowing the system to retrieve more accurate and context-specific information. By handling these subqueries separately, the agent can generate more complete and relevant responses.

In [35]:
# Add this code cell at the end of the notebook

print("--- Running Agentic RAG Examples ---")

# Example 1: Query expected to route to OPENAI_QUERY
#print("\\n" + "="*30 + " Example 1: OpenAI Query " + "="*30)
#agentic_rag("What is the purpose of the OpenAI moderation API?")

# Example 2: Query expected to route to 10K_DOCUMENT_QUERY (and find data)
#print("\\n" + "="*30 + " Example 2: 10K Query (Success) " + "="*30)
# Use Lyft 2024 data which exists in the Qdrant DB provided
#agentic_rag("What was Lyft's revenue growth rate in 2024 compared to 2023?")

# Example 3: Query expected to route directly to INTERNET_QUERY
#print("\\n" + "="*30 + " Example 3: Direct Internet Query " + "="*30)
#agentic_rag("What are the latest advancements in renewable energy technology?")

# Example 4: Query expected to route to 10K_DOCUMENT_QUERY, fail, and fallback to INTERNET_QUERY
print("\\n" + "="*30 + " Example 4: 10K Query (Fallback) " + "="*30)
# Use Uber 2024, which is NOT in the Qdrant DB provided (only Uber 2021 is)
# This assumes retrieve_and_response correctly returns DATA_NOT_FOUND_SIGNAL
agentic_rag("What was Uber's revenue in 2024?")

# Example 5: Compound Query testing sub-query splitting and potential fallback
print("\\n" + "="*30 + " Example 5: Compound Query (Sub-queries + Fallback) " + "="*30)
agentic_rag("Explain the OpenAI Assistants API and what was Lyft revenue in 2024?")

# Example 6: Another compound query mixing successful 10K and fallback 10K
print("\\n" + "="*30 + " Example 6: Compound Query (Mixed 10K Success/Fallback) " + "="*30)
agentic_rag("What was Lyft revenue in 2024 and what was Uber revenue in 2024?")


print("\\n" + "="*30 + " Examples Complete " + "="*30)

--- Running Agentic RAG Examples ---
\n============================== Example 4: 10K Query (Fallback) ==============================
👤 User Query: What was Uber's revenue in 2024?

📍 Selected Route: INTERNET_QUERY
📝 Reason: No financial data for uber 2024 in database
⚙️ Processing query...

Getting your response from the internet 🌐 ...
🤖 BOT RESPONSE:

Uber's revenue in 2024 was approximately $44 billion. This figure represents an 18% increase from the previous year, 2023. 

Here are some additional details regarding Uber's financial performance in 2024:

1. **Quarterly Breakdown**:
   - **Q1 2024**: Revenue of $3.2 billion.
   - **Q2 2024**: Revenue of $3.3 billion.
   - **Q3 2024**: Revenue of $6.4 billion.
   - **Q4 2024**: Revenue of $9.936 billion.

2. **Gross Bookings**: For the full year, Uber reported gross bookings of approximately $37.575 billion.

3. **Income from Operations**: Uber generated an income from operations of $652 million for the year.

4. **Year-over-Year Growth